In [2]:
import numpy as np
import onnxruntime as ort
from PIL import Image
def preprocess_image(image_path, input_size=(224, 224)):
    image = Image.open(image_path)
    image = image.convert('RGB')
    image = image.resize(input_size)
    image_array = np.array(image)
    image_array = image_array / 255.0
    image_array = image_array.astype(np.float32)
    image_array = np.transpose(image_array[np.newaxis, ...], (0,3,1,2))
    return image_array

def run_inference(image_path, model_path):
    session = ort.InferenceSession(model_path)
    input_name = session.get_inputs()[0].name
    input_data = preprocess_image(image_path)
    outputs = session.run(None, {input_name: input_data})
    return outputs[0], outputs[1]


In [4]:
import os
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import numpy as np
# 이미지가 있는 기본 디렉토리 경로
base_dir = "/workspace/a-eye-lab-research/export/testset"
model_path = "/workspace/a-eye-lab-research/export/output/eye_detection.onnx"

# 결과를 저장할 리스트 초기화
y_true = []
y_pred = []

# 모든 하위 디렉토리(0, 1)를 순회
for class_dir in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_dir)
    if os.path.isdir(class_path):
        print(f"\nProcessing class directory: {class_dir}")
        
        # 각 클래스 디렉토리 내의 모든 이미지 파일을 순회
        for image_file in os.listdir(class_path):
            if image_file.endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(class_path, image_file)
                
                # 추론 실행
                result, similarity = run_inference(image_path, model_path)
                
                # 실제 클래스와 예측 결과 저장
                y_true.append(int(class_dir))
                y_pred.append(1 if result else 0)

                print(f"\nImage: {image_file}")
                print(f"True class: {class_dir}")
                print("=== 이미지 유사성 판단 ===")
                print(
                    f"눈 이미지, {np.round(similarity,2)}"
                    if result
                    else f"눈 이미지 아님, {np.round(similarity,2)}"
                )

# 성능 지표 계산
cm = confusion_matrix(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
specificity = cm[0,0] / (cm[0,0] + cm[0,1])  # TN / (TN + FP)

print("\n=== 성능 평가 결과 ===")
print("\n컨퓨전 매트릭스:")
print(cm)
print(f"\n정밀도(Precision): {precision:.3f}")
print(f"재현율(Recall): {recall:.3f}")
print(f"특이도(Specificity): {specificity:.3f}")


Processing class directory: 0

Image: 000000581020.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.46]

Image: 000000578321.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.49]

Image: 000000581281.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.52]

Image: 000000581823.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.46]

Image: 000000581003.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.51]

Image: 000000581908.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.49]

Image: 000000581358.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.48]

Image: 000000581645.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.48]

Image: 000000581333.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.44]

Image: 000000581672.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.5]

Image: 000000580948.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.45]

Image: 000000580594.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0.41]

Image: 000000581515.jpg
True class: 0
=== 이미지 유사성 판단 ===
눈 이미지 아님, [0